## This python notebook has examples of the Set-Coverage Problem and Partial Set-Coverage Problem
### load libraries

In [85]:
import geopandas as gpd
import pandas as pd
import os, sys, time
sys.path.append(r'C:\Code\Github\GOST_PublicGoods\GOSTNets\GOSTNets')
import GOSTnet as gn
import importlib
import networkx as nx
import osmnx as ox
from shapely.ops import unary_union
from shapely.wkt import loads
from shapely.geometry import LineString, MultiLineString, Point

## calculate OD Matrix

In [39]:
#%time OD = gn.calculate_OD(G, origins, destinations, fail_value = 9999999999999)

In [86]:
OD_df = pd.read_csv(r'C:\Users\gost_\Desktop\lima\data\OD_time\output_gt_origins_destinations.csv', header=0, index_col=0)

In [87]:
#need to cast columns as int
OD_df.columns = OD_df.columns.astype(int)

In [88]:
OD_df.keys()

Int64Index([112655, 106515, 104469,  79897,  43040,  90146, 102438,  71719,
              8237,  28719,
            ...
             32700,  47038,  73664,  63424, 118731,  12246,  45018,  45023,
             32754,  34811],
           dtype='int64', length=375)

In [64]:
#OD_df.index

Int64Index([ 28677,  96264,  18441,  88074,  45067,  47117,  53262, 110611,
             40980,  83989,
            ...
             26590,  18396,  16350,  26588,  14308,  34789, 108522,  63467,
             59386,  83967],
           dtype='int64', length=634)

In [67]:
#OD_df.iloc[2,2]

840.315034912666

In [68]:
#OD_df.loc[88074]

24420    388.215409
29476    646.804176
87562    528.362960
89721    633.018070
47038    302.963681
34559    117.647717
Name: 88074, dtype: float64

In [69]:
#OD_df.loc[88074,'87562']
#test
OD_df.loc[88074,87562]

528.3629602062092

In [89]:
OD_df[0:10]

,112655,106515,104469,79897,43040,90146,102438,71719,8237,28719,...,32700,47038,73664,63424,118731,12246,45018,45023,32754,34811
1,11626.587080,8446.211658,8998.629429,7361.393123,12711.278749,10321.899165,9447.117396,5575.375228,11317.168966,11951.335959,...,11429.874273,9021.442455,12274.145550,12173.464021,11583.088864,9678.818275,6462.276334,7900.901985,15763.682622,6979.723873
3,4464.791907,7182.204605,6525.147239,9549.588199,4910.268512,5644.988485,5723.902088,9665.088617,7864.899348,10435.472353,...,7585.613803,5929.613237,5002.962716,4949.309405,7768.070153,5534.176071,8488.779357,8705.900383,13012.925368,9647.753117
4,1660.797699,3441.941622,2784.884257,5688.006885,2584.156017,1538.779946,1983.639106,5924.825634,4124.636365,6695.209371,...,4237.341672,2189.350254,2287.151760,1304.308323,4393.687366,1793.913088,4748.516375,4965.637401,9272.662386,5907.490135
5,1908.715596,4966.454625,4309.397260,7333.838220,309.292218,3737.208407,3511.187419,7449.338637,3884.551867,8219.722374,...,2690.392682,3713.863257,2041.794552,3273.229404,2872.849033,3318.426091,6273.029378,6490.150404,10797.175389,7432.003138
8,6481.620185,3301.244764,3853.662535,2017.213150,7566.311855,5176.932270,4302.150502,2438.672351,6172.202072,7256.105010,...,6284.907379,3876.475561,7129.178656,7028.497127,6438.121970,4533.851381,1540.407201,2870.930627,10618.715728,878.520713
11,4495.598611,7553.337641,6896.280275,9920.721235,3712.714116,6090.472727,6097.490168,10036.221653,7549.931717,10806.605389,...,6355.772533,6300.746273,4714.546092,5626.493723,6538.228883,5905.309107,8859.912393,9077.033419,13384.058404,10018.886153
13,7772.480585,4592.105163,5144.522935,3507.286628,8857.172254,6467.792670,5593.010902,1721.268733,7463.062472,8097.229464,...,7575.767779,5167.335960,8420.039056,8319.357527,7728.982370,5824.711781,2608.169840,4046.795491,11909.576128,3125.617378
16,4168.501141,1209.054575,1760.654904,2846.104220,5253.192811,2863.813226,1842.786696,3185.031145,3895.708035,5164.594632,...,4008.413342,1777.570944,4816.059612,4465.520716,4164.759036,2221.357971,2011.602268,2235.342396,8524.602395,3165.055228
18,9174.791428,7133.341723,6311.025605,9966.371587,10259.483097,7870.103513,7000.152828,9964.050219,7991.796211,7896.014347,...,8104.501518,6574.477887,9822.349898,9721.668370,8257.716109,7227.010058,8787.740959,8952.012672,661.404793,9946.714719
22,4443.009761,2399.503298,1643.535364,5233.643238,5527.701430,3138.321846,2268.371161,5231.321869,3260.014544,3160.661364,...,3372.719851,1842.696219,5090.568231,4989.886702,3525.934442,2495.228391,4055.012610,4218.174246,5398.195071,5213.986370


In [121]:
#for each row get min value
OD_df_min = OD_df.min(axis=1)

In [122]:
OD_df_min[0:10]

1     3527.657334
3      787.857397
4      549.809191
5      150.223406
8      679.815777
11    1319.701820
13     186.442778
16     159.305852
18     252.397205
22     250.617796
dtype: float64

In [124]:
OD_df_min_max = OD_df_min.max(axis=0)

In [125]:
#This should be the worst service time for an origin in the whole dataset
OD_df_min_max

6881.0824891589355

In [110]:
OD_df.index

Int64Index([     1,      3,      4,      5,      8,     11,     13,     16,
                18,     22,
            ...
            119619, 119629, 119630, 119635, 119638, 119639, 119640, 119644,
            119645, 119646],
           dtype='int64', length=54314)

In [111]:
OD_df.keys()

Int64Index([112655, 106515, 104469,  79897,  43040,  90146, 102438,  71719,
              8237,  28719,
            ...
             32700,  47038,  73664,  63424, 118731,  12246,  45018,  45023,
             32754,  34811],
           dtype='int64', length=375)

In [22]:
#OD_df = pd.DataFrame(OD, columns = destinations, index = origins)

In [118]:
import importlib
importlib.reload(gn)

peartree version: 0.6.1 
networkx version: 2.2 
matplotlib version: 3.0.3 
osmnx version: 0.9 


<module 'GOSTnet' from 'C:\\Code\\Github\\GOST_PublicGoods\\GOSTNets\\GOSTNets\\GOSTnet.py'>

## Set-Coverage Problem
### Objective: Determine the minimum number of facilities and their locations in order to cover all demands within a pre-specified maximum distance (or time) coverage

### inputs include a pre-specified maximum distance coverage of 3000 seconds

In [119]:
     set_coverage_result = gn.optimize_set_coverage(OD_df,max_coverage = 1800)

number of origins
54314
print totalCoveredFacilities
53065
print percent coverage
97.70040873439629
print prob obj
0*__dummy


In [126]:
set_coverage_result2 = gn.optimize_set_coverage(OD_df,max_coverage = 6882)

number of origins
54314
print totalCoveredFacilities
54314
print percent coverage
100.0
print prob obj
X_100807 + X_100919 + X_101517 + X_101696 + X_101743 + X_102129 + X_102148 + X_102271 + X_102438 + X_102665 + X_102987 + X_102992 + X_103520 + X_103523 + X_103565 + X_103721 + X_103926 + X_103943 + X_104084 + X_104266 + X_104378 + X_104469 + X_10499 + X_105266 + X_105714 + X_105919 + X_105959 + X_106409 + X_106515 + X_106758 + X_107330 + X_107457 + X_107741 + X_107826 + X_107977 + X_108771 + X_10891 + X_110522 + X_111150 + X_111365 + X_111866 + X_11198 + X_112218 + X_112331 + X_112655 + X_112747 + X_11305 + X_113613 + X_113664 + X_114081 + X_114214 + X_114339 + X_114538 + X_114844 + X_114927 + X_115198 + X_115325 + X_115641 + X_116003 + X_116107 + X_11631 + X_117527 + X_118244 + X_118274 + X_1184 + X_118731 + X_118940 + X_119296 + X_119547 + X_12246 + X_12395 + X_12499 + X_12757 + X_12824 + X_1309 + X_13617 + X_13888 + X_1402 + X_14418 + X_145 + X_14559 + X_14940 + X_15524 + X_15915 +

In [127]:
set_coverage_result2

[2382, 24420, 30135, 3427]

In [113]:
set_coverage_result

[100807,
 100919,
 101517,
 101696,
 101743,
 102129,
 102148,
 102271,
 102438,
 102665,
 102987,
 102992,
 103520,
 103523,
 103565,
 103721,
 103926,
 103943,
 104084,
 104266,
 104378,
 104469,
 10499,
 105266,
 105714,
 105919,
 105959,
 106409,
 106515,
 106758,
 107330,
 107457,
 107741,
 107826,
 107977,
 108771,
 10891,
 110522,
 111150,
 111365,
 111866,
 11198,
 112218,
 112331,
 112655,
 112747,
 11305,
 113613,
 113664,
 114081,
 114214,
 114339,
 114538,
 114844,
 114927,
 115198,
 115325,
 115641,
 116003,
 116107,
 11631,
 117527,
 118244,
 118274,
 1184,
 118731,
 118940,
 119547,
 12246,
 12395,
 12499,
 12757,
 12824,
 1309,
 13617,
 13888,
 1402,
 14418,
 145,
 14559,
 14940,
 15524,
 15915,
 16196,
 16673,
 17095,
 17441,
 18348,
 19271,
 19319,
 19592,
 19711,
 19793,
 19986,
 20314,
 20391,
 20990,
 21027,
 21418,
 216,
 21740,
 22046,
 22156,
 22348,
 22667,
 22790,
 22814,
 22971,
 23057,
 2307,
 23102,
 23466,
 23668,
 2382,
 24176,
 24420,
 24428,
 24463,
 24

In [114]:
len(set_coverage_result)

373

In [107]:
OD_df[set_coverage_result]

,100807,100919,101517,101696,101743,102129,102148,102271,102438,102665,...,97120,97960,98130,98389,98505,9887,98884,99251,99597,99605
1,6609.203045,10655.738571,12676.399912,11162.737495,7567.212061,9750.870107,6315.300653,11919.918352,9447.117396,11582.525925,...,6628.411448,10128.164330,11366.126591,7983.572613,9266.659583,13399.962290,9906.274809,11532.287039,11754.040521,12857.615262
3,9570.916985,4865.256028,4421.734148,4095.747613,8911.401950,6992.150307,8736.996974,4630.126481,5723.902088,8134.053577,...,8751.911906,5886.469042,8615.369337,7505.945479,6849.665486,5131.880523,5661.185257,5228.571098,4110.779115,10106.858008
4,5830.654002,757.672612,2315.582963,881.510238,5171.138968,3251.887325,4996.733991,1932.924562,1983.639106,4393.790594,...,5011.648924,1824.111567,4875.106354,3399.736939,3109.402503,3039.145340,1554.976718,1772.996121,1097.193445,6366.595025
5,7355.167005,2961.567385,935.378459,2214.179718,6695.651971,4776.400327,6521.246994,1614.120012,3511.187419,3351.890088,...,6536.161927,3673.754373,6399.619357,5292.127667,4633.915506,1658.944280,3537.703122,3011.996548,2473.235974,7471.229673
8,1672.988513,5509.011029,7531.433018,6017.770601,1154.201767,4605.903213,1513.644434,6774.951458,4302.150502,6437.559031,...,1706.542315,4983.197436,6221.159696,2722.650162,4121.692689,8254.995395,4761.307915,6387.320145,6609.073627,7712.648368
11,9942.050021,5314.831705,3038.893694,4567.444037,9282.534986,7363.283343,9108.130010,4286.871552,6097.490168,7017.269939,...,9123.044942,6260.057122,8986.502373,7879.010683,7220.798522,3104.848328,6106.669499,5598.879564,4826.500294,10477.991044
13,2755.096551,6801.632076,8822.293418,7308.631000,3713.105566,5896.763613,2461.194159,8065.811858,5593.010902,7728.419431,...,2774.304954,6274.057836,7512.020096,4129.466118,5412.553089,9545.855795,6052.168314,7678.180545,7899.934027,9003.508767
16,3088.219095,2796.213197,5218.313974,3704.651557,2428.704061,2511.789880,2256.939503,4461.832414,1842.786696,4164.862264,...,2273.488024,2670.078392,4127.046364,557.834274,2028.685057,5941.876351,2448.188870,4074.201101,4078.339451,5618.535035
18,9869.878587,8316.511291,10224.604260,8710.941843,9210.363552,6425.497352,9035.958576,9468.122700,7000.152828,8257.153170,...,9050.873508,7676.368679,4784.168299,7820.980962,6665.076498,10948.166638,7454.479157,9080.491387,9302.244869,5971.397990
22,5137.150237,3584.729624,5492.822593,3979.160176,4477.635203,1693.715685,4303.230227,4736.341033,2268.371161,3525.371503,...,4318.145159,2944.587011,1000.639040,3089.199295,1997.586257,6216.384971,2722.697490,4348.709720,4570.463202,2492.127711


## Partial Set-Coverage Problem
### Objective: Determine the minimum number of facilities and their locations in order to cover a given fraction of the population within a pre-specified maximum distance (or time) coverage

### we need to produce a series that has each origin and its respective population

In [73]:
origins_w_demands_series = pd.Series(origins_gdf.demand.values,index=origins_gdf.NN)

AttributeError: 'GeoDataFrame' object has no attribute 'demand'

### inputs include covering 90 percent of the population, a pre-specified maximum distance coverage of 2000 seconds, and a  series of origins with their population

In [115]:
partial_set_coverage_result = gn.optimize_partial_set_coverage(OD_df, pop_coverage = .9, max_coverage = 2000, origins_pop_series = origins_w_demands_series, existing_facilities = None)

print min_coverage
12419780.4


In [112]:
partial_set_coverage_result

[15739, 1646, 2440, 25486, 33112, 4092, 4965, 6724, 7011, 8994]